<a href="https://colab.research.google.com/github/rama100/python-notebooks/blob/main/lightning-intro-to-argparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Easily process arguments for your command-line script with `argparse`

**Author**: [Rama Ramakrishnan](https://ramakrishnan.com)

You have written a nice Python script that can be run on the command-line. It expects a bunch of arguments. Inside the script, you need to "receive" these arguments and do things with them. 

`argparse` is a Python module that makes this "receiving" easy. 

I will use the following example to demonstrate. I will assume that your script expects four arguments as shown below.

```
python your_script.py \
 --model gpt-4o \
 --temperature 0.0 \
 --prompt "hello world" \
 --open_api_key sk ... 
```

Note: `argparse` can do all kinds of cool things and the official [tutorial](https://docs.python.org/3/howto/argparse.html) is a wonderful resource to learn about these things. But for a common use-case like the one above, you just need to know the bare minimum. Let's go!!


**Step 0**: Import it.

```
import argparse
```

**Step 1**: Create an `ArgumentParser` object with a helpful description.

```
parser = argparse.ArgumentParser("python my_script.py")
```

**Step 2**: Add arguments one by one with the `add_argument` method. Since our example above has 4 arguments, we will do this 4 times.

```
parser.add_argument("--model", 
                    type=str, 
                    default="gpt-4o-mini",
                    help="the OPENAI model you want the script to use")
parser.add_argument("--prompt", 
                    type=str, 
                    default="The cat sat on the ",
                    help="the prompt you want to pass into the model")
parser.add_argument("--openai_api_key", 
                    type=str, 
                    required=True,
                    help="Your API access key")
parser.add_argument("--temperature", 
                    type=float, 
                    default=0.0,
                    help="Closer to 1.0 for creative, closer to 0.0 for deterministic")
```

In each line:
* the first parameter e.g., `--model` names the argument 
* `type` denotes the type of argument. The first three arguments are `str` and the last one - `temperature` - is `float`
* you can specify a `default` value. I have done so for 3 of the 4 arguments.
* you can indicate that an argument MUST be supplied with `required=True`. I have done so for the "--openai_api_key" argument.
* you can provide an optional `help` string. This will get printed out if the user makes a mistake or wants help.


**Step 3**: Finally, we add this line to the script. 

```
args = parser.parse_args()
```

This line does the heavy lifting: it grabs all the arguments typed by the user and puts in the right places in the `args` object. 

The `args` object will have attributes that exactly match the arguments:
* `model`
* `prompt`
* `openai_api_key`
* `temperature`

The **values** of these attributes will be set to what the user typed in for the arguments. 

With `args` in hand, you can do **whatever you want** in your code. As a concrete example, I will simply print out the attributes of `args`.

```
d_args = vars(args)
for k,v in d_args.items():
    print(f"Name: {k}, Value: {v}")
```

OK, let's collect all the lines above into a script called `my_script.py`. 

In [1]:
%%writefile my_script.py
import argparse
parser = argparse.ArgumentParser("python my_script.py")
parser.add_argument("--model", 
                    type=str, 
                    default="gpt-4o-mini",
                    help="the OPENAI model you want the script to use")
parser.add_argument("--prompt", 
                    type=str, 
                    default="The cat sat on the ",
                    help="the prompt you want to pass into the model")
parser.add_argument("--openai_api_key", 
                    type=str, 
                    required=True,
                    help="Your API access key")
parser.add_argument("--temperature", 
                    type=float, 
                    default=0.0,
                    help="Closer to 1.0 for creative, closer to 0.0 for deterministic")
args = parser.parse_args()


d_args = vars(args)
for k,v in d_args.items():
    print(f"Name: {k}, Value: {v}")

Overwriting my_script.py


OK, let's try running it as a user.

In [2]:
!python3 my_script.py \
 --model gpt-4.5 \
 --prompt "what is the capital of paris" \
 --openai_api_key 'test key' \
 --temperature 0.8

Name: model, Value: gpt-4.5
Name: prompt, Value: what is the capital of paris
Name: openai_api_key, Value: test key
Name: temperature, Value: 0.8


As promised, the `args` object: 
* has 4 attributes
* the names of the attributes are the argument names
* the values are what we passed in for those arguments when we ran the script

Another example: Just for fun, let's say we just want to print out the value of the `temperature` argument 5 times, but increasing it by 10 each time.

We will add the following code snippet to our script:
```
for i in range(5):
    print(f"The value of temperature is {args.temperature + (10 * i)}")
```

In [3]:
%%writefile my_script.py
import argparse
parser = argparse.ArgumentParser("python my_script.py")
parser.add_argument("--model", 
                    type=str, 
                    default="gpt-4o-mini",
                    help="the OPENAI model you want the script to use")
parser.add_argument("--prompt", 
                    type=str, 
                    default="The cat sat on the ",
                    help="the prompt you want to pass into the model")
parser.add_argument("--openai_api_key", 
                    type=str, 
                    required=True,
                    help="Your API access key")
parser.add_argument("--temperature", 
                    type=float, 
                    default=0.0)
args = parser.parse_args()

d_args = vars(args)
for k,v in d_args.items():
    print(f"Name: {k}, Value: {v}")
for i in range(5):
    print(f"The value of temperature is {args.temperature + (10 * i)}")


Overwriting my_script.py


Let's run it again.

In [4]:
!python3 my_script.py \
 --model gpt-4.5 \
 --prompt "what is the capital of paris" \
 --openai_api_key 'test key' \
 --temperature 0.8

Name: model, Value: gpt-4.5
Name: prompt, Value: what is the capital of paris
Name: openai_api_key, Value: test key
Name: temperature, Value: 0.8
The value of temperature is 0.8
The value of temperature is 10.8
The value of temperature is 20.8
The value of temperature is 30.8
The value of temperature is 40.8


----

**Bottomline: you can process the attributes of `args` any way you want.**

----

Next, let's talk about what is probably `argparse`'s biggest benefit: if the user makes a mistake with the arguments, `argparse` will catch it and warn the user. You don't have to deal with this inside your script.


To demonstrate, let's see what happens if the user doesn't supply the `required` OPENAI_API_KEY argument.

In [5]:
!python3 my_script.py \
 --model gpt-4.5 \
 --prompt "what is the capital of paris" \
 --temperature 0.8

usage: python my_script.py [-h] [--model MODEL] [--prompt PROMPT]
                           --openai_api_key OPENAI_API_KEY
                           [--temperature TEMPERATURE]
python my_script.py: error: the following arguments are required: --openai_api_key


Notice this line:
>error: the following arguments are required: --openai_api_key

Also, there's a helpful 'usage' message.

----
What if the user uses the wrong **type** of input? Let's see what happens if the user types in text rather than a number for the `temperature` argument.

In [6]:
!python3 my_script.py \
 --model gpt-4.5 \
 --prompt "what is the capital of paris" \
 --openai_api_key 'test key' \
 --temperature rama

usage: python my_script.py [-h] [--model MODEL] [--prompt PROMPT]
                           --openai_api_key OPENAI_API_KEY
                           [--temperature TEMPERATURE]
python my_script.py: error: argument --temperature: invalid float value: 'rama'


 `argparse` has caught the error. Notice this line:
 >error: argument --temperature: invalid float value: 'rama'

----
Another example: The user uses a name that doesn't match any of the 4 arguments we specified. Instead of `prompt`, we will say `question`.

In [213]:
!python3 my_script.py \
 --model gpt-4.5 \
 --question "what is the capital of paris" \
 --openai_api_key 'test key' \
 --temperature 0.8

usage: python my_script.py [-h] [--model MODEL] [--prompt PROMPT]
                           --openai_api_key OPENAI_API_KEY
                           [--temperature TEMPERATURE]
python my_script.py: error: unrecognized arguments: --question what is the capital of paris


`argparse` has caught the error. Notice this line:
 >error: error: unrecognized arguments: --question what is the capital of paris

----
One final thing: if the user runs the script with -h or --help, `argparse` will automatically generate a nice help message. This is where those optional `help` strings (in the `parser.add_argument` code) get used.

In [7]:
!python3 my_script.py -h

usage: python my_script.py [-h] [--model MODEL] [--prompt PROMPT]
                           --openai_api_key OPENAI_API_KEY
                           [--temperature TEMPERATURE]

optional arguments:
  -h, --help            show this help message and exit
  --model MODEL         the OPENAI model you want the script to use
  --prompt PROMPT       the prompt you want to pass into the model
  --openai_api_key OPENAI_API_KEY
                        Your API access key
  --temperature TEMPERATURE


----
That's it. Try it out and good luck!